In [1]:
project_directory = "/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13"
import sys
sys.path.append(project_directory)

In [3]:
import carla
import yaml
import os

from utils_cogmod.server_utils import CarlaServerManager

In [4]:
# file_path = os.path.join(project_directory, 'outputs', '2024-04-26', '14-41-15')
# diagnostics = os.path.join(file_path, 'diagnostics')

In [5]:
def read_config(config_path):
    with open(config_path, 'r') as file:
        cfg = yaml.safe_load(file)
    return cfg

def init_client(host, port):
    try:
        client = carla.Client(host, port)
        print(f"client connected to {host}:{port}")
        print(f"client.get_server_version(): {client.get_server_version()}, client.get_client_version(): {client.get_client_version()}")
        client.set_timeout(60.0)
    except RuntimeError as re:
        if "timeout" not in str(re) and "time-out" not in str(re):
            print("Could not connect to Carla server because:", re)
        client = None 
    return client

# Assuming 'client' is already initialized and is an instance of carla.Client
# The path to your recorder log file

def run_replay(config_path, recorder_file_path, actor_id, start_time, duration):
    cfg = read_config(config_path)
    # server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
    # server_manager.stop()
    # server_manager.start()

    client = init_client(cfg['host'], cfg['port'])
    # Check if file exists
    try:
        with open(recorder_file_path, 'r') as file:
            print(f"File found: {recorder_file_path}")
    except FileNotFoundError:
        print(f"File not found: {recorder_file_path}")
        exit()
    try:
        # Corrected call to replay_file
        replay_result = client.replay_file(
            recorder_file_path,
            time_start=start_time,  # Explicitly specifying as double
            duration=duration,    # Explicitly specifying as double
            follow_id=actor_id,     # This is already an int, but making sure it's clearly specified
            replay_sensors=True  # Assuming you want to replay sensors, adjust as necessary
        )
        print("Replaying recorded log file...", replay_result)
    except Exception as e:
        print(f"An error occurred while trying to replay the file: {e}")

In [6]:
config_path = r"/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/config/benchmark.yaml"
recorder_file_path = r"/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log"

In [7]:
cfg = read_config(config_path)
server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
server_manager.stop()
server_manager.start()

In [8]:
client = init_client(cfg['host'], cfg['port'])

all_info = client.show_recorder_file_info(recorder_file_path, True)

client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13


In [ ]:
# # save all info in a text file 
# with open(os.path.join(r"C:\Users\abjawad\Documents\GitHub\carla-roach-0.9.13\notebooks", 'all_info.txt'), 'w') as f:
#     f.write(all_info)

In [9]:
collision_info = client.show_recorder_collisions(recorder_file_path, "h", "v")

print(type(collision_info))
print(collision_info)

<class 'str'>
Version: 1
Map: Town04
Date: 04/28/24 01:42:28

    Time  Types     Id Actor 1                                 Id Actor 2                            
     174   v v    4763 vehicle.lincoln.mkz_2017              4929 vehicle.tesla.cybertruck           

Frames: 2375
Duration: 237 seconds



/home/ubuntu/vihaan-devel/carla/CARLA_0.9.13/CarlaUE4.sh: line 5: 12600 Killed                  "$UE4_PROJECT_ROOT/CarlaUE4/Binaries/Linux/CarlaUE4-Linux-Shipping" CarlaUE4 "$@"


In [ ]:
start_time = 130
duration = 0
actor_id = 227
other_actor_id = 287
run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

In [ ]:
world = client.get_world() 
actor = world.get_actor(actor_id)
actor.destroy()